# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [61]:
from pyspark.sql import SparkSession

In [62]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [63]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [64]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [65]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [66]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [67]:
from pyspark.ml.feature import VectorAssembler

In [68]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], outputCol='features')

In [69]:
output = assembler.transform(data)

In [70]:
final_data = output.select('features', 'churn')

In [71]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [72]:
from pyspark.ml.classification import LogisticRegression

In [73]:
lr_churn = LogisticRegression(labelCol='churn')

In [74]:
fitted_churn_model = lr_churn.fit(train_churn)

In [75]:
training_sum = fitted_churn_model.summary

In [76]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                608|                608|
|   mean| 0.1611842105263158|0.11513157894736842|
| stddev|0.36800353520708334|0.31944347578802185|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [77]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [78]:
pred_labels = fitted_churn_model.evaluate(test_churn)

In [79]:
pred_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.36632209572696...|[0.99828448008835...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.55242140409991...|[0.82526318271309...|       0.0|
|[28.0,11128.95,1....|    0|[3.97335537887362...|[0.98153707969871...|       0.0|
|[28.0,11245.38,0....|    0|[3.85846683175139...|[0.97933569839658...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.18501400759869...|[0.98500624215584...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.80817128232725...|[0.99190331791658...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.52402070914391...|[0.98927102925772...|       0.0|
|[29.0,11274.46,1....|    0|[4.22421987668763...|[0.98557439522654...|       0.0|
|[29.0,13255.05,1....|    0|[3.82809844554170...|[0.97871209495860...|       0.0|
|[30.0,13473.35,

In [80]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [81]:
auc = churn_eval.evaluate(pred_labels.predictions)

In [82]:
auc

0.7780448717948718

predict on new data


In [83]:
final_lr_model = lr_churn.fit(final_data)

In [84]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [85]:
test_new_customers = assembler.transform(new_customers)

In [86]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [87]:
test_new_customers = assembler.transform(new_customers)

In [88]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [90]:
final_results = final_lr_model.transform(test_new_customers)

In [91]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [92]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          